<a href="https://colab.research.google.com/github/Lanzero225/Python-Notebooks/blob/main/Philippine%20Weather/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Philippine Weather Analysis (2024-2025)

The Philippines is a tropical country located in Southeast Asia, situated near the equator and along the western Pacific Ocean. Due to its geographic location, the country experiences highly variable weather conditions and is frequently affected by tropical storms and typhoons.

This analysis aims to examine historical weather data from the past two years across different locations in the Philippines. By analyzing key weather variables such as temperature, weather conditions, and other attributes, this study seeks to identify patterns, seasonal trends, and extreme weather occurrences. The insights derived from this analysis may help in understanding the country’s weather behavior and its implications for disaster preparedness, urban planning, and climate-related decision-making.



# Data Importing

To begin first, I will import the necessary libraries to begin this analysis.

In [1]:
import pandas as pd
import numpy as np

The dataset used for this analysis uses a dataset from Kaggle:
- https://www.kaggle.com/datasets/bwandowando/philippine-major-cities-weather-data
- https://www.kaggle.com/datasets/bwandowando/philippine-major-cities-weather-data-2025

The data here is gathered from:
- https://openweathermap.org

The 2024 dataset gathers records in 3-hour intervals whilst the 2025 dataset gathers records in 1-hour intervals. In the upcoming steps, data enrichment will be implemented to remove that inconsistency.

First, let us begin by loading the dataset. I uploaded the dataset to my personal repository for easy access, which can be accessed with:
- "https://raw.githubusercontent.com/Lanzero225/Python-Notebooks/main/Philippine%20Weather/Data/{YYYY-MM}_CombinedData.csv"

After which, the csv file will be converted into a DataFrame and aggregated into a singular DataFrame.

In [2]:

directories = pd.period_range('2023-11', '2025-12', freq='M').strftime('%Y%m').tolist()
dfs = []

for i in directories:
    url = (
        "https://raw.githubusercontent.com/"
        "Lanzero225/Python-Notebooks/main/"
        f"Philippine%20Weather/Data/{i}_CombinedData.csv"
    )
    dfs.append(pd.read_csv(url))

weather_dataframe = pd.concat(dfs, ignore_index=True)

Looking initially into the code, we see that there are columns with missing values and mismatched datatypes.

Missing Columns:
- visibility
- coord.lon
- coord.lat
- main.sea_level
- main.grnd_level
- wing.gust
- weather.id
- weather.icon
- extraction_date_time
- sys.id
- sys.type
- rain.1h

Datatype Mismatch:
- datetime
- extraction_date_time
- sys.sunruse
- sys.sunset

In [3]:
weather_dataframe.info(show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513792 entries, 0 to 1513791
Data columns (total 27 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   visibility            1512148 non-null  float64
 1   datetime              1513792 non-null  object 
 2   coord.lon             447165 non-null   float64
 3   coord.lat             447165 non-null   float64
 4   main.temp             1513792 non-null  float64
 5   main.feels_like       1513792 non-null  float64
 6   main.temp_min         1513792 non-null  float64
 7   main.temp_max         1513792 non-null  float64
 8   main.pressure         1513792 non-null  float64
 9   main.humidity         1513792 non-null  float64
 10  main.sea_level        1472164 non-null  float64
 11  main.grnd_level       1472164 non-null  float64
 12  wind.speed            1513792 non-null  float64
 13  wind.deg              1513792 non-null  float64
 14  wind.gust             1295449 non-

In [5]:
weather_dataframe['sys.sunrise']

,sys.sunrise
0,2023-11-06 05:58:51+08:00
1,2023-11-06 05:55:14+08:00
2,2023-11-06 05:53:22+08:00
3,2023-11-06 05:40:04+08:00
4,2023-11-06 05:53:01+08:00
...,...
1513787,2025-12-31 05:59:57+08:00
1513788,2025-12-31 06:20:48+08:00
1513789,2025-12-31 06:05:34+08:00
1513790,2025-12-31 06:28:25+08:00


## Data Definition

Let's dive deep into each of the columns in this dataset.

These are the columns that refer to location and time:
- datetime - Date and time of current tracked weather.
- coord.lon - Longitude coordinate of city.
- coord.lat - Latitude coordinate of city.
- city_name - Name of the city.
  - e.g Alaminos, 119.9811, 16.1561
- sys.sunrise - Date and time of a city's sunrise
- sys.sunset - Date and timeof a city's sunset
  - e.g 2023-11-06 05:58:51+08:00

The following describe the basic weather signal units:
- visibility - Measure of distance at which objects can be discerned (0 to 10 km).
- main.temp - Temperature in Celsius.
- main.feels_like - Human perception of temperature in Celsius.
- main.pressure/main.sea_level - Atmospheric pressure on the sea level, hPa.
= main.grnd_level - Atmospheric pressure on the ground level, hPa
- main.humidity - Percentage of humidity.
- main.temp_min - Minimum temperature at the moment.
- main.temp_max - Maximum temperature at the moment.
- wind.speed - Wind speed in meter/sec (m/s).
- wind.deg - Wind direction in meteorological degrees
- wind.gust - Wind gust in meter/sec (m/s).
- clouds.all - Percentage of cloudiness
- rain.1h - Pecipitation in milimmeter/hour (mm/h).

The following describe basic weather labels:
- weather.main - Refers to the category of weather for that record.
- weather.description - Refers to the subtype of weather under that record.
- weather.id - Integer value referring to the ID corresponding to the specific weather description.
  - 201 -> thunderstorm with rain

Lastly, the following refer to the system data that is used while extracting data:
- sys.id - Internal parameter
- sys.type - Internal parameter
- extraction_date_time - Date and timestamp of when the system extracted the record.
- weather.icon - Corresponding icon associated with weather.description.

In [ ]:
columns = list(weather_dataframe.columns)
weather_dataframe[['main.pressure', 'main.sea_level']]

,main.pressure,main.sea_level
0,1009.0,1009.0
1,1010.0,NaN
2,1010.0,1010.0
3,1009.0,1009.0
4,1009.0,NaN
...,...,...
1513787,1009.0,1009.0
1513788,1010.0,1010.0
1513789,1008.0,1008.0
1513790,1010.0,1010.0


In [7]:
categorical_weather_labels = ['weather.main','weather.description','weather.id','weather.icon']
time_location_labels = ['datetime','sys.sunrise','sys.sunset','coord.lat','coord.lon','city_name']
weather_signal_labels = [
    'main.temp','main.feels_like','main.temp_min','main.temp_max',
    'main.pressure','main.humidity','main.sea_level','main.grnd_level',
    'wind.speed','wind.deg','wind.gust','clouds.all','visibility','rain.1h'
]
system_data = ['sys.type','sys.id','extraction_date_time', 'weather.icon']

In [ ]:
weather_dataframe.drop(system_data, inplace=True, axis=1)

In [ ]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   visibility           1512148 non-null  float64
 1   datetime             1513792 non-null  object 
 2   coord.lon            447165 non-null   float64
 3   coord.lat            447165 non-null   float64
 4   main.temp            1513792 non-null  float64
 5   main.feels_like      1513792 non-null  float64
 6   main.temp_min        1513792 non-null  float64
 7   main.temp_max        1513792 non-null  float64
 8   main.pressure        1513792 non-null  float64
 9   main.humidity        1513792 non-null  float64
 10  main.sea_level       1472164 non-null  float64
 11  main.grnd_level      1472164 non-null  float64
 12  wind.speed           1513792 non-null  float64
 13  wind.deg             1513792 non-null  float64
 14  wind.gust            1295449 non-null  float64
 15

In [ ]:
time_labels = ['datetime', 'sys.sunrise', 'sys.sunset', 'extraction_date_time']
for col in time_labels:
    if col in weather_dataframe.columns:
        weather_dataframe[col] = pd.to_datetime(weather_dataframe[col], errors='coerce')


In [ ]:
weather_dataframe[['coord.lat', 'coord.lon']] = (
    weather_dataframe.groupby('city_name')[['coord.lat', 'coord.lon']]
      .transform(lambda x: x.fillna(x.mean()))
)


In [ ]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1512148 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1472164 non-null  flo

In [ ]:
cities = list(weather_dataframe['city_name'])
print("There are", len(cities), "cities.")
print("The first five are:", cities[:5])

There are 1513792 cities.
The first five are: ['Alaminos', 'Angeles City', 'Antipolo', 'Bacolod', 'Bacoor']


In [ ]:
def interpolate_columns_by_city(df, columns, method='linear'):
    df = df.sort_values(['city_name', 'datetime'])
    for col in columns:
        df[col] = (
            df.groupby('city_name')[col]
              .transform(lambda s: s.interpolate(method=method, limit_direction='both'))
        )
    return df

weather_dataframe = interpolate_columns_by_city(
    weather_dataframe,
    ['main.sea_level', 'main.grnd_level', 'wind.gust', 'visibility']
)

In [ ]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1513792 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1513792 non-null  float64 

In [ ]:
weather_dataframe[(weather_dataframe['weather.id'].isnull())][['weather.id', 'weather.main', 'weather.description']]

,weather.id,weather.main,weather.description
447165,NaN,Clouds,overcast clouds
447303,NaN,Clouds,broken clouds
447441,NaN,Clouds,broken clouds
447579,NaN,Rain,light rain
447717,NaN,Clouds,broken clouds
...,...,...,...
1513246,NaN,Clouds,broken clouds
1513380,NaN,Clouds,broken clouds
1513517,NaN,Clouds,scattered clouds
1513655,NaN,Clouds,broken clouds


In [ ]:
weather_dataframe['weather.description'].unique()

array(['overcast clouds', 'light rain', 'scattered clouds', 'clear sky',
       'few clouds', 'broken clouds', 'moderate rain',
       'heavy intensity rain', 'very heavy rain', 'haze',
       'light intensity shower rain', 'fog', 'mist', 'thunderstorm',
       'thunderstorm with light rain', 'thunderstorm with heavy rain',
       'thunderstorm with rain', 'light thunderstorm',
       'heavy intensity shower rain', 'shower rain', 'smoke',
       'light intensity drizzle', 'extreme rain', 'drizzle'], dtype=object)

In [ ]:
weather_code_dict = {
    200: 'thunderstorm with light rain',
    201: 'thunderstorm with rain',
    202: 'thunderstorm with heavy rain',
    210: 'light thunderstorm',
    211: 'thunderstorm',
    500: 'light rain',
    501: 'moderate rain',
    502: 'heavy intensity rain',
    503: 'very heavy rain',
    504: 'extreme rain',
    520: 'light intensity shower rain',
    521: 'shower rain',
    522: 'heavy intensity shower rain',
    531: 'ragged shower rain',
    300: 'light intensity drizzle',
    301: 'drizzle',
    302: 'heavy intensity drizzle',
    310: 'light intensity drizzle rain',

    701: 'mist',
    711: 'smoke',
    721: 'haze',
    741: 'fog',


    800: 'clear sky',
    801: 'few clouds',
    802: 'scattered clouds',
    803: 'broken clouds',
    804: 'overcast clouds'
}

In [ ]:
description_to_id_dict = {v: k for k, v in weather_code_dict.items()}
weather_dataframe['weather.id'] = weather_dataframe.apply(
    lambda row:
        description_to_id_dict.get(row['weather.description']) if pd.isna(row['weather.id']) else row['weather.id'],
    axis=1
)


In [ ]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1513792 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1513792 non-null  float64 

In [ ]:
weather_dataframe['rain.1h'].fillna(0, inplace=True)
weather_dataframe.info()

/tmp/ipython-input-972124671.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather_dataframe['rain.1h'].fillna(0, inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1513792 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1513792 non-null  float64 